In [33]:
import pandas as pd

df = pd.read_csv('Corona_NLP_train.csv', encoding = "ISO-8859-1")



In [10]:
df

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative
...,...,...,...,...,...,...
41152,44951,89903,"Wellington City, New Zealand",14-04-2020,Airline pilots offering to stock supermarket s...,Neutral
41153,44952,89904,NaN,14-04-2020,Response to complaint not provided citing COVI...,Extremely Negative
41154,44953,89905,NaN,14-04-2020,You know itÂs getting tough when @KameronWild...,Positive
41155,44954,89906,NaN,14-04-2020,Is it wrong that the smell of hand sanitizer i...,Neutral


In [34]:
df_new = df[['OriginalTweet', 'Sentiment']]

In [14]:
# baixando as libs para pre processamento

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

nltk.download('stopwords')

stopwords = set(stopwords.words('english'))

#stemmer = PorterStemmer()
nltk.download("wordnet")
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/developer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/developer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [46]:
# pre processamento

    # Remover caracteres especiais e números
    #text = re.sub('[^A-Za-z\s\']+', '', text)

def preprocess_text(text):

    padrao_arroba = re.compile(r'@\S+')
      # Remover as strings com "@" no início da string
    text = re.sub(padrao_arroba, '', text)

        #remover sites
    #pattern_url = re.compile(r'https?:\/\/.*?[\s+]')

    text = re.sub(r'http\S+', '', text)
    

    # Converter tudo para letras minúsculas
    text = text.lower()

    
    # Remover as tags HTML
    text = re.sub('<[^<]+?>', '', text)    
    
    
    # Tokenizar o texto em palavras
    words = nltk.word_tokenize(text)

    # remover palavras que so possua caractere especial
    words = [word for word in words if not re.match('^[^A-Za-z0-9]+$', word)]

    # remover palavras que tenha tamanho 1
    words = [word for word in words if len(word) > 2]

    
    # Remover as stopwords
    words = [word for word in words if word not in stopwords]
    
    # Aplicar o stemming
    #words = [stemmer.stem(word) for word in words]
    words = [lemmatizer.lemmatize(word) for word in words]

    # Juntar as palavras em uma única string
    text = ' '.join(words)
  
    return text

In [27]:
# rodar o pre processamento

def preprocess_collection(docs):

    preprocessed_documents = []

    #for document in docs:
    for index, row in docs.iterrows():
        tema = row['Sentiment']
        #nome_arquivo = document['nome_arquivo']
        conteudo = row['OriginalTweet']
        
        preprocessed_conteudo = preprocess_text(conteudo)
        
        preprocessed_document = {
            'tema': tema,
        #    'nome_arquivo': nome_arquivo,
            'conteudo': preprocessed_conteudo
        }
        
        preprocessed_documents.append(preprocessed_document)
        
    return preprocessed_documents

In [39]:
df_new.head(4)

,OriginalTweet,Sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,advice Talk to your neighbours family to excha...,Positive
2,Coronavirus Australia: Woolworths to give elde...,Positive
3,My food stock is not the only one which is emp...,Positive


In [47]:
preprocessed_documents = preprocess_collection(df_new)


In [48]:
preprocessed_documents

[{'tema': 'Neutral', 'conteudo': ''},
 {'tema': 'Positive',
  'conteudo': 'advice talk neighbour family exchange phone number create contact list phone number neighbour school employer chemist set online shopping account po adequate supply regular med order'},
 {'tema': 'Positive',
  'conteudo': 'coronavirus australia woolworth give elderly disabled dedicated shopping hour amid covid-19 outbreak'},
 {'tema': 'Positive',
  'conteudo': "food stock one empty please n't panic enough food everyone take need stay calm stay safe covid19france covid_19 covid19 coronavirus confinement confinementotal confinementgeneral"},
 {'tema': 'Extremely Negative',
  'conteudo': "ready supermarket covid19 outbreak paranoid food stock litteraly empty coronavirus serious thing please n't panic cause shortage coronavirusfrance restezchezvous stayathome confinement"},
 {'tema': 'Positive',
  'conteudo': 'news regionâ\x92s first confirmed covid-19 case came sullivan county last week people flocked area store pu

In [49]:
frases = [o['conteudo'] for o in preprocessed_documents]

In [50]:
from rank_bm25 import BM25Okapi

tokenized_corpus = [doc.split(" ") for doc in frases]

bm25 = BM25Okapi(tokenized_corpus)

In [63]:
#dataset de teste

import pandas as pd

df_test = pd.read_csv('Corona_NLP_test.csv', encoding = "ISO-8859-1")
df_test_new = df_test[['OriginalTweet', 'Sentiment']]

In [68]:
df_test_sorted = df_test_new.sort_values('Sentiment')

In [69]:
data_test = preprocess_collection(df_test_sorted)

In [89]:
data_test[3800]

IndexError: list index out of range

In [71]:
# apenas para teste
tokenized_query = data_test[101]['conteudo'].split(" ")
bm25.get_top_n(tokenized_query, frases, n=100)

['agree president mean give fuck 250,000 american life american healthcare system general stock price comcast delta airline whole point covid2019 trumpvirus',
 "trump n't know saudi trying kill competition low oil price including american oil production covid2019 coronavirus covid19 trump trumpvirus trumpviruscoverup trumppressconference cnn",
 "'ve come conclusion supermarket customer powys mid- wale could n't give coronavirus covid19 yet another one coughing hand rearranging stock decides chocolate egg want buy",
 'legacy known president america renamed coronavirus trumpvirus president full america ran toiletpaper',
 'day ago grocery store older woman saying didnâ\x92t get grocery earlier president said worry donald trump failed county underprepared inaction american pay price coronavirus',
 'trying get crowd grocery store eye random last loaf who-knows-what-kind bread one else seemed want corona coronavirus coronapocalypse trumpvirus',
 'world waking covid2019 president trump warns 

In [90]:
array_temas = ['Extremely Negative',
'Extremely Positive',
'Negative',
'Neutral',
'Positive']
temas = {}

for d in array_temas:
    temas[d] = {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}

print(temas)    

{'Extremely Negative': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'Extremely Positive': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'Negative': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'Neutral': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'Positive': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}}


In [93]:
from statistics import mean

from IPython.display import clear_output

# consultando
p10 = []
p20 = []
p50 = []
p100 = []
map_f = []
tema_atual = ""

for c, doc_consulta in enumerate(data_test):
    clear_output(wait=True)
    print(c)
    #ranking = similaridade(doc_consulta['conteudo'], preprocessed_documents, idfs)
    tokenized_query = doc_consulta['conteudo'].split(" ")
    ranking = bm25.get_top_n(tokenized_query, frases, n=len(data_test))
    acertos = 0
    
    #map_f.append(calcula_map(ranking, doc_consulta['tema']))
    
    for i, doc in enumerate(ranking):
        #print(f"Documento {doc[0]} - Target {doc[1]} -->  {doc[2]} - Similaridade: {doc[3]}")
        #if doc[1] == doc_consulta['tema']:
        #print(doc_consulta['tema'])
        if doc_consulta['tema'] in doc:
            #print(f"{doc_consulta['tema']} e {doc}")
            acertos+=1
        if i+1 == 10:
            p10.append(acertos/10.0)            
        if i+1 == 20:
            p20.append(acertos/20.0)
        if i+1 == 50:
            p50.append(acertos/50.0)
        if i+1 == 100:
            p100.append(acertos/100.0) 
        #map.append(calcula_map(ranking, doc_consulta['tema']))
    if c > 0:
        if tema_atual != doc_consulta['tema'] or c == len(data_test) - 1:
            temas[tema_atual]['p10'] = mean(p10)
            temas[tema_atual]['p20'] = mean(p20)
            temas[tema_atual]['p50'] = mean(p50)
            temas[tema_atual]['p100'] = mean(p100)
            temas[tema_atual]['map'] = mean(map_f)
            p10 = []
            p20 = []
            p50 = []
            p100 = []
            #map = []

    tema_atual = doc_consulta['tema']
    if c > 202:
        break    


47


KeyboardInterrupt: 

In [92]:
temas

{'Extremely Negative': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0},
 'Extremely Positive': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0},
 'Negative': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0},
 'Neutral': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0},
 'Positive': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}}